# Long Short-Term Memory (LSTM) Autoecoder (AE) Anomaly Detection

## How to run a Juypter notebook

https://docs.jupyter.org/en/latest/running.html

## Including dependencies

To run this, you'll need matplotlib, seaborn, torch, sklearn, numpy, and pandas. Pip should handle these for you, but torch can be tricky. Google is your friend here

In [1]:
# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Torch
import torch
import torch.nn as nn
import torch.utils.data as data_utils

# sklean
from sklearn import preprocessing

# Misc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Setting our execution device

This variable allows the use of the GPU (if avaliable). This only works with NVIDIA GPUs

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cuda


### Encoder Class

Below is the class that contains the encoder part of our network. It is a 2 layer encoder that uses two LSTM (RNNs) as its layers

In [3]:
class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))   # (batch_size, seq_len, num of features)
    # print(x)
    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)

    return hidden_n.reshape((1, self.embedding_dim))  # num of units/neurons into the hidden layer
  
  # passing in results from the encoder

### Decoder Class

Below is the class that contains the decoder part of our network. It is a 2 layer encoder that uses two LSTM (RNNs) as its layers

In [4]:
class Decoder(nn.Module):

  def __init__(self, seq_len, input_dim=64, n_features=1):

    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim

    self.hidden_dim, self.n_features = 2 * input_dim, n_features

    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

    self.output_layer = nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):

    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))

    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)

    x = x.reshape((self.seq_len, self.hidden_dim))

    return self.output_layer(x)

### LSTM-Autoencoder class

Now, we can combine our encoder and decoder networks into a single LSTM-AE. Notice, in this class, the forward function only makes calls to the encoder then the decoder.

In [5]:
class RecurrentAutoencoder(nn.Module):

  def __init__(self, seq_len, n_features=1, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

## Reading in our synthetic data

In [6]:
# Raw data from .csv
df = pd.read_csv('synthetic_data_normal.csv')
print(df)

# Grab the targets (anomaly labels)
# Not needed for the LSTM-AE, but useful for further classification
targets_raw = df.iloc[:, 0]
targets_np = targets_raw.to_numpy()
le = preprocessing.LabelEncoder()
targets = le.fit_transform(targets_np)
inputs  = torch.tensor(targets)

# Grab the features (time-series)
features_raw = df.iloc[:, 1:]
features_np = features_raw.to_numpy()
features_total = np.column_stack([targets,features_np])
features = torch.Tensor(features_total)

# Do a little extra work to get the number of sequences, length of each sequence, and number of features
sequences = features_raw.astype(np.float32).to_numpy().tolist()
dataset = [torch.tensor(s).unsqueeze(1) for s in features_np]
n_seq, seq_len, n_features = torch.stack(dataset).shape

# Print out some metrics of our dataset
print("\nNumber of sequences:", n_seq)
print("Sequence length: ", seq_len)
print("Number of features: ", n_features)

# Create a tensor dataset and a dataloader
train = data_utils.TensorDataset(features, inputs)

# Probably want to leave batch_size equal to 1 here...
train_loader = data_utils.DataLoader(train, batch_size=1, shuffle=True)

          0       0.1         1         2         3         4         5  \
0    normal -0.348796 -0.085726  0.309447  0.597268  0.601409  0.712171   
1    normal  0.034529 -0.006321  0.075302  0.303938  0.564451  0.624538   
2    normal -0.098415  0.140292  0.223849  0.194160  0.469268  0.791196   
3    normal  0.088932 -0.013717  0.388148  0.433235  0.302517  0.441382   
4    normal  0.253196 -0.099384  0.083056  0.197967  0.432012  0.094493   
..      ...       ...       ...       ...       ...       ...       ...   
995  normal -0.275241  0.232920  0.405765  0.404851  0.533835  0.256463   
996  normal  0.219121  0.040536  0.009555  0.117296  0.501165  0.609651   
997  normal -0.237998  0.070051  0.353420  0.560810  0.505321  0.162667   
998  normal -0.189628 -0.078464  0.160853  0.502834  0.182108  0.902421   
999  normal  0.084698  0.267118  0.065285 -0.023995  0.233047  0.318685   

            6         7         8  ...       140       141       142  \
0    0.646340  0.627614  0.

### Instantiate a model of LSTM-AE network

Here, we create an object to store the network model. Then we move the model to the GPU (if avaliable). Finally, we print out the network architecture.

In [7]:
# the third parameter (embedding dim) is one you may want to try increasing if you have more complex data and are getting poor results
model = RecurrentAutoencoder(seq_len+1, n_features, 32)    # model obj
model = model.to(device)                                   # moving model to gpu

print(model)

RecurrentAutoencoder(
  (encoder): Encoder(
    (rnn1): LSTM(1, 64, batch_first=True)
    (rnn2): LSTM(64, 32, batch_first=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(32, 32, batch_first=True)
    (rnn2): LSTM(32, 64, batch_first=True)
    (output_layer): Linear(in_features=64, out_features=1, bias=True)
  )
)


### Training function

In [8]:
def train_model(model, train_dataset, n_epochs):

  # Try different learning rates! Does that change how well the network changes?
  optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)

  # Try different loss functions! See why (or why not) they work better
  # Remember: The notation of Loss and Cost functions are often used interchangeably
  criterion = nn.L1Loss(reduction='sum').to(device)     # Loss function
                                                        # summing error; L1Loss = mean absolute error in torch

  history = dict(train=[], val=[])  
  for epoch in range(1, n_epochs + 1):
    model = model.train()                     # Putting the model in training mode

    train_losses = []                         # list to store loss values

    for seq_true,_ in train_dataset:          # iterate over each seq for train data
      optimizer.zero_grad()                   # no accumulation

      seq_true = seq_true.to(device)          # putting sequence to gpu
      seq_pred = model(seq_true)              # prediction

      loss = criterion(seq_pred, seq_true.T)  # measuring error

      loss.backward()                         # backprop
      optimizer.step()                        # optimize weights

      train_losses.append(loss.item())        # record loss by adding to training losses

    train_loss = np.mean(train_losses)        # get the mean loss
    history['train'].append(train_loss)       # store mean loss
    
    print(f'Epoch {epoch}: train loss = {train_loss}')

  return history 

### Train the model for n epochs

An epoch in a neural network is the training of the neural network with all the training data for one cycle. Lower loss is better.

In [ ]:
# More epochs take longer to train but could provide better results on more complex data or larger models
epochs = 25
his = train_model(model, train_loader, epochs)

# Saving the model as a file
MODEL_PATH = 'ltsm_ae_model_small.pth'
torch.save(model, MODEL_PATH)

Epoch 1: train loss = 141.85227192115784
Epoch 2: train loss = 28.986126922607422
Epoch 3: train loss = 26.219880439758303
Epoch 4: train loss = 26.076440423965455
Epoch 5: train loss = 26.023223121643067
Epoch 6: train loss = 25.928776906967162
Epoch 7: train loss = 25.595530435562132
Epoch 8: train loss = 25.64119337463379
Epoch 9: train loss = 25.39769501876831
Epoch 10: train loss = 25.490263862609865
Epoch 11: train loss = 25.396369943618776
Epoch 12: train loss = 25.300572896957398
Epoch 13: train loss = 25.23149860191345
Epoch 14: train loss = 25.114116025924684
Epoch 15: train loss = 25.055405590057372
Epoch 16: train loss = 25.025915613174437
Epoch 17: train loss = 24.962692499160767
Epoch 18: train loss = 25.01837215423584
Epoch 19: train loss = 25.05641528892517
Epoch 20: train loss = 25.067996810913087
Epoch 21: train loss = 24.841528827667236
Epoch 22: train loss = 24.962525718688966


In [ ]:
# Just some plotting stuffs
sns.set_theme()
ax = plt.figure().gca()
ax.plot(his['train'])
# ax.plot(his['val'])
plt.ylabel('Loss') 
plt.xlabel('Epoch')
plt.legend(['train'])
# plt.legend(['train', 'test'])
plt.title('Loss over training epochs')
plt.show()

In [ ]:
def predict(model, dataset):
  predictions, losses = [], []
  criterion = nn.L1Loss(reduction = 'sum').to(device) #L1 to gpu

  with torch.no_grad():
    model = model.eval()      # putting in evaluation mode          
    for seq_true,_ in dataset:
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

      loss = criterion(seq_pred, seq_true.T)  # calculating loss
      
      predictions.append(seq_pred.cpu().numpy().flatten()) # appending predictions & loss to results 
      losses.append(loss.item())
  return predictions, losses

### Run a prediction using the trained model and normal dataset

Running a prediction means that we do a single forward propagation of the network and take the result of the output layer as the answer

In [ ]:
model = torch.load('ltsm_ae_model.pth')     # Load model
model = model.to(device)                    # put model on GPU
_, losses = predict(model, train_loader)    # Get prediction, losses

Now we can grab a histogram of our losses. We want this so we can define a decision boundary we can use to classify what qualifies as a normal time-series. Since we ran a prediction based on the dataset we know contains only "normal" time-series, the threshold must contain the values within the histogram

In [ ]:
# Histogram of losses
sns.histplot(
    losses, kde=True,
    stat="density", kde_kws=dict(cut=30),
    alpha=.4, edgecolor=(1, 1, 1, .4),
)
plt.show()

In [ ]:
# Threshold of what we consider to be normal
# For normal time-series, we want our losses to capture values below the threshold
# The concept of a threshold here is a little arbitary
# Try playing with the logic here and see if you can get a better view of the results!
THRESHOLD = 42
correct = sum(l <= THRESHOLD for l in losses)
print(f'Correct normal predictions: {correct}/{len(train_loader)}')

### Run a prediction using the trained model and anomaly dataset

In [ ]:
df = pd.read_csv('synthetic_data_high.csv')
targets_raw = df.iloc[:, 0]
targets_np = targets_raw.to_numpy()
le = preprocessing.LabelEncoder()
targets = le.fit_transform(targets_np)
inputs  = torch.tensor(targets)

features_raw = df.iloc[:, 1:]
features_np = features_raw.to_numpy()
features_total = np.column_stack([targets,features_np])
features = torch.Tensor(features_total)

sequences = features_raw.astype(np.float32).to_numpy().tolist()
dataset = [torch.tensor(s).unsqueeze(1) for s in features_np]
n_seq, seq_len, n_features = torch.stack(dataset).shape

anomaly = data_utils.TensorDataset(features, inputs)
anomaly_loader = data_utils.DataLoader(anomaly, batch_size=1, shuffle=True)

In [ ]:
_, losses2 = predict(model, anomaly_loader)

In [ ]:
sns.histplot(
    losses2, kde=True,
    stat="density", kde_kws=dict(cut=20),
    alpha=.4, edgecolor=(1, 1, 1, .4),
)
plt.show()

In [ ]:
# Threshold of what we consider to be an anomaly
# For normal time-series, we want our losses to capture values above the threshold
# The concept of a threshold here is a little arbitary
# Try playing with the logic here and see if you can get a better view of the results!
THRESHOLD2 = 46
correct = sum(l > THRESHOLD2 for l in losses2)
print(f'Correct anomaly precitions: {correct}/{len(anomaly_loader)}')

### Overlay truth and reconstructions

This plots the first 6 normal time-series + reconstructions and the first 6 anomaly time-series + reconstructions

In [ ]:
def plot_prediction(data, model, title, ax):
  predictions, pred_losses = predict(model, [data])
  
  ax.plot(data[0][0], label= 'true')
  ax.plot(predictions[0], label= 'predicted')
  ax.grid()
  ax.set_title(f'{title} (loss: {np.around(pred_losses, 2)})') # showing loss in the title with 2 decimal points
  ax.legend()

  # 1st row is 6 examples from normal heartbeat, next row represents anomalies
fig, axs = plt.subplots(
    nrows= 2,
    ncols= 4,
    sharex = True,
    sharey = True,
    figsize = (16, 6)
)

# looping through 6 examples
train_iter = iter(train_loader)
for i in range(4):
    data_batch = next(train_iter)
    plot_prediction(data_batch, model, title= 'Normal', ax = axs[0, i])
    
anomaly_iter = iter(anomaly_loader)
for i in range(4):
    data_batch = next(anomaly_iter)
    plot_prediction(data_batch, model, title= 'Anomaly', ax = axs[1, i])

fig.tight_layout()
plt.show()